In [ ]:
!pip install webdriver-manager

In [5]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

from IPython.display import clear_output

import re

import math
from datetime import datetime

In [ ]:
latin_stress_to_standard = {
    "á": "a",
    "é": "e",
    "í": "i",
    "ó": "o",
    "ú": "u",
}

### 20minutos

In [ ]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
JOURNAL_URL = "https://www.20minutos.es/"

In [ ]:
DEFAULT_WAIT_TIME = 5
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=None)
driver.get(JOURNAL_URL)
clickable_condition = EC.element_to_be_clickable((By.XPATH, "//button/span[contains(text(), 'Aceptar ')]"))
WebDriverWait(driver, DEFAULT_WAIT_TIME).until(clickable_condition).click()

Extract hrefs

In [ ]:
areas_href = driver.find_elements(By.XPATH, "//ul[@class='default-menu']/li//a")
areas_name = [a.get_attribute("href").split("/")[-2] for a in areas_href]
areas_name

In [64]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
JOURNAL_URL = "https://www.20minutos.es/"
JOURNAL_URL = "https://okdiario.com/"

In [ ]:
latin_stress_to_standard = {
    "á": "a",
    "é": "e",
    "í": "i",
    "ó": "o",
    "ú": "u",
}

In [66]:
DEFAULT_WAIT_TIME = 5
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=None)
driver.get(JOURNAL_URL)
clickable_condition = EC.element_to_be_clickable((By.XPATH, "//button/span[contains(text(), 'Aceptar ')]"))
WebDriverWait(driver, DEFAULT_WAIT_TIME).until(clickable_condition).click()

In [ ]:
areas_href = driver.find_elements(By.XPATH, "//ul[@class='default-menu']/li//a")
areas_name = [a.get_attribute("href").split("/")[-2] for a in areas_href]
areas_name

In [ ]:
all_page_urls = driver.find_elements(By.XPATH, "//a[@href]")

In [ ]:
all_page_urls

### OKdiario

In [ ]:
DEFAULT_WAIT_TIME = 5
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=None)
driver.get(JOURNAL_URL)
clickable_condition = EC.element_to_be_clickable((By.XPATH, "//button/span[contains(text(), 'Aceptar ')]"))
WebDriverWait(driver, DEFAULT_WAIT_TIME).until(clickable_condition).click()

Extract hrefs

In [175]:
def extract_articles_meta():
    all_articles = driver.find_elements(By.XPATH, "//article[@id]")
    all_page_ids = [elem.get_attribute("id") for elem in all_articles]
    all_page_urls = [elem.find_element(By.XPATH, "/a[@href]/header/h2").text for elem in all_articles]
    all_page_titles = [elem.find_element(By.XPATH, "/a/header[@class='article-header']") for elem in all_articles]
    all_page_author_date = [elem.find_element(By.XPATH, "/a/footer[@class='article-footer']").text for elem in all_articles]
    return all_page_ids, all_page_urls, all_page_titles, all_page_author_date
def clean_urls_root(urls, root="okdiario.com"):
    urls = pd.Series(urls)
    urls.dropna(inplace=True)
    urls_root = urls[urls.str.contains(root)]
    return urls_root

In [76]:
all_page_urls_elements = driver.find_elements(By.XPATH, "//a[@href]")
all_page_urls = [elem.get_attribute("href") for elem in all_page_urls_elements]
len(all_page_urls), all_page_urls[:5]

(396,
 ['https://okdiario.com/',
  'https://okdiario.com/ultimas-noticias/',
  'https://okdiario.com/espana/',
  'https://okdiario.com/madrid/',
  'https://okdiario.com/cataluna/'])

Analyze urls

In [154]:
urls = pd.Series(all_page_urls)
urls.dropna(inplace=True)
urls.head()

0                     https://okdiario.com/
1    https://okdiario.com/ultimas-noticias/
2              https://okdiario.com/espana/
3              https://okdiario.com/madrid/
4            https://okdiario.com/cataluna/
dtype: object

In [162]:
# Extract urls with base root
is_base_root = urls.str.contains("okdiario.com")
print(is_base_root.sum(), "roots")
urls_root = urls[is_base_root]
urls_root.head(12)

0                          https://okdiario.com/
1         https://okdiario.com/ultimas-noticias/
2                   https://okdiario.com/espana/
3                   https://okdiario.com/madrid/
4                 https://okdiario.com/cataluna/
5                https://okdiario.com/andalucia/
6               https://okdiario.com/pais-vasco/
7     https://okdiario.com/comunidad-valenciana/
8                  https://navarra.okdiario.com/
9                 https://okdiario.com/baleares/
10                https://okdiario.com/economia/
11        https://okdiario.com/economia/consumo/
dtype: object

In [157]:
# Extract non-root levels
urls_nonroot = urls_root.str.extract(("//okdiario.com/(.*)"), expand=False)
urls_nonroot.str.split("/").head(20)

0                                         []
1                       [ultimas-noticias, ]
2                                 [espana, ]
3                                 [madrid, ]
4                               [cataluna, ]
5                              [andalucia, ]
6                             [pais-vasco, ]
7                   [comunidad-valenciana, ]
8                                        NaN
9                               [baleares, ]
10                              [economia, ]
11                     [economia, consumo, ]
12                      [economia, empleo, ]
13                         [investigacion, ]
14                      [diariomadridista, ]
15    [diariomadridista, ultimas-noticias, ]
16         [diariomadridista, real-madrid, ]
17                              [deportes, ]
18                      [deportes, futbol, ]
19             [deportes, atletico-madrid, ]
dtype: object

In [161]:
# Frequencies of non-root levels in urls
nonroot_levels_freq = urls.str.split("/").str.len()
print(nonroot_levels_freq.value_counts())
levels_gt_2 = urls[nonroot_levels_freq > 2]
levels_gt_2

5    287
6     92
7      9
4      7
8      1
dtype: int64


0                       https://okdiario.com/
1      https://okdiario.com/ultimas-noticias/
2                https://okdiario.com/espana/
3                https://okdiario.com/madrid/
4              https://okdiario.com/cataluna/
                        ...                  
391    https://okdiario.com/politica-cookies/
392            https://okdiario.com/archivos/
393            https://okdiario.com/noticias/
394                 https://okdiario.com/feed
395       https://okdiario.com/quienes-somos/
Length: 396, dtype: object

#### In-depth case

In [169]:
deep_1_url = "https://okdiario.com/espana/"

In [170]:
DEFAULT_WAIT_TIME = 5
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=None)
driver.get(deep_1_url)
clickable_condition = EC.element_to_be_clickable((By.XPATH, "//button/span[contains(text(), 'Aceptar ')]"))
WebDriverWait(driver, DEFAULT_WAIT_TIME).until(clickable_condition).click()

In [176]:
deep_1_urls = extract_articles_meta()
len(deep_1_urls)

MaxRetryError: HTTPConnectionPool(host='localhost', port=45637): Max retries exceeded with url: /session/7e100674bb96c649192dd348ec1e1b8c/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000025E57963700>: Failed to establish a new connection: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión'))

### Xataka

In [182]:
def xataka_extract_articles_meta(last_count: int) -> dict:
    #d, m, y = date.split("-")
    d, m, y = datetime.today().strftime("%d-%m-%Y").split("-")
    d = d[-1] if d[0] == "0" else d
    m = m[-1] if m[0] == "0" else m
    xataka_url_archives_1 = f"https://www.xataka.com/archivos/{y}/{m}"
    date_fmt = f"{d} de {int_to_month_sp[m]} de {y}"
    search_command_str = f"//div[@class='list-archive']/h2[contains(text(), '{date_fmt}')]"
    DEFAULT_WAIT_TIME = 5
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=None)
    driver.get(xataka_url_archives_1)
    clickable_condition = EC.element_to_be_clickable((By.XPATH, "//button/span[contains(text(), 'Aceptar ')]"))
    WebDriverWait(driver, DEFAULT_WAIT_TIME).until(clickable_condition).click()
    on_new_articles = False
    try:
        date_exists = driver.find_elements(By.XPATH, search_command_str)
        articles_in_date = driver.find_elements(By.XPATH, f"//div[@class='list-archive']/ul[1]/li/a")
        len_articles_in_date = len(articles_in_date)
        if len_articles_in_date > last_count:
            on_new_articles = True
    except:
        print("Error during article extraction")
    if on_new_articles:
        output = {}
        new_id = last_count
        # Reverse, because extraction started from newest article. Bring only new ones
        new_articles_in_date = articles_in_date[::-1][last_count:len_articles_in_date]
        for i in range(len_articles_in_date - last_count):
            article_elem = new_articles_in_date[i]
            href = article_elem.get_attribute("href")
            title = article_elem.text
            output[new_id] = {"url": href, "title": title}
            new_id += 1
        return output
    else:
        return {}

In [185]:
new_articles = xataka_extract_articles_meta(38)
new_articles

{38: {'url': 'https://www.xataka.com/magnet/hay-miles-prorrusos-mundo-asi-que-rusia-les-esta-construyendo-ciudad-vivan-alli',
  'title': 'Hay miles de prorrusos por el mundo. Así que Rusia les está construyendo una ciudad para que vivan allí'},
 39: {'url': 'https://www.xataka.com/seleccion/estos-auriculares-bluetooth-marshall-cancelacion-ruido-tienen-descuento-se-quedan-baratos-que-nunca-amazon',
  'title': 'Estos auriculares Bluetooth Marshall con cancelación de ruido tienen descuento y se quedan más baratos que nunca en Amazon'},
 40: {'url': 'https://www.xataka.com/espacio/inframundo-galactico-rincon-confines-via-lactea-al-que-viajan-estrellas-al-morir',
  'title': 'El "inframundo galáctico", el rincón en los confines de la Vía Láctea al que viajan las estrellas al morir'},
 41: {'url': 'https://www.xataka.com/makers/decktility-minipc-basado-raspberry-pi-que-recuerda-a-tiempos-dorados-pda-palm',
  'title': 'Decktility es el miniPC basado en Raspberry Pi que recuerda a los tiempos d